# 1. 패키지 불러오기

In [83]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from tqdm import tqdm

# 2. 데이터 불러오기

In [3]:
load_path = './data/' # 데이터 불러오는 경로
save_path = './preprocessing/' # 전처리 이후 데이터 저장하는 경로

In [5]:
train_err = pd.read_csv(load_path + 'train_err_data.csv')
train_qual = pd.read_csv(load_path + 'train_quality_data.csv')
train_prob = pd.read_csv(load_path + 'train_problem_data.csv')

/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel_30969/3504990764.py:2: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  train_qual = pd.read_csv(load_path + 'train_quality_data.csv')


In [6]:
test_err = pd.read_csv(load_path + 'test_err_data.csv')
test_qual = pd.read_csv(load_path + 'test_quality_data.csv')

/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel_30969/3785495537.py:2: DtypeWarning: Columns (4,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  test_qual = pd.read_csv(load_path + 'test_quality_data.csv')


# 3. 데이터 분석
## Dataset 정보

### Err dataset
Err Data는 사람들이 에러를 접한 시간을 기준으로
어떤 Model & Fwver을 사용하고, 접한 Errtype & Errcode는 무엇인지에 대하여 알려주는 데이터이다.
- User ID : 사용자 고유 ID
- Time : 에러 발생 시간
- Model : 에러가 발생한 모델명
- Fwver : 에러가 발생한 펌웨어 버전
- Errtype : 에러 분류 (에러 타입)
- Errcode : 어떤 에러가 발생하였는지 (에러 코드)

In [13]:
train_err.info() # column user_id, time, model_nm, fwver, errtype, errcode

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [14]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 756.8+ MB


### Qual dataset
Quality Data는 사용자의 시스템 작동 중 문제가 발생하면
측정 가능한 지표들로 해당 시점으로부터 2시간 단위로 수집한 정보를 알려주는 데이터이다.
- User ID : 사용자 고유 ID
- Time : 퀄리티가 수집되기 시작한 시간
- Fwver : 퀄리티 시작 시점 기준의 펌웨어 버전
- Quality : 에러 퀄리티 수치 (0~12, 총 13개 컬럼)

In [16]:
train_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [17]:
test_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB


### Problem dataset
Problem Data는 전체 사용자 중에서, 불만을 제기한 사람들의 시간을 알려주는 데이터이다.
- User ID : 불만을 제기한 유저
- Time : 불만을 제기한 시간
- 한 사용자가 여러번 불만을 제기할 수 있음

In [18]:
train_prob.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


## 결측치 확인

### Train Err Data 결측치 확인

In [19]:
train_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [20]:
train_err[train_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN


In [28]:
train_err[(train_err['user_id']==13639) & (train_err['model_nm']=='model_2') & (train_err['fwver']=='04.33.1261') & (train_err['errtype']==5) & (train_err['time']==20201121191718)]

,user_id,time,model_nm,fwver,errtype,errcode
3825744,13639,20201121191718,model_2,04.33.1261,5,NaN
3825745,13639,20201121191718,model_2,04.33.1261,5,40013


- 같은 시간, 같은 펌웨어, 같은 유저, 같은 에러타입의 경우 같은 에러코드를 가질 확률이 높다
고로 결측치를 대체할 수 있다.

- 동일한 에러코드로 교체한다.

In [29]:
train_err['errcode'] = train_err['errcode'].fillna('40013')

### Train Qual Data 결측치 확인

- Fwver 결측치 확인 
- quality_0, quality_2, quality_5  결측치 존재

In [32]:
train_qual.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [40]:
train_qual[train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

55.45762711864407

In [41]:
train_qual[~train_qual['fwver'].isnull()].groupby('user_id')['time'].unique().to_frame()['time'].str.len().mean()

7.9863813229571985

In [42]:
fwms_idx = train_qual[train_qual['fwver'].isnull()].index
train_qual = train_qual.drop(fwms_idx)

In [44]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].fillna(train_qual[i].mode(0)[0])

### Test Err Data 결측치


In [45]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [46]:
test_err[test_err['errcode'].isnull()]

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,20201115044317,model_2,04.33.1261,5,NaN
4038892,33681,20201103110259,model_2,04.33.1185,5,NaN
9486881,38991,20201127213838,model_2,04.33.1261,5,NaN
10425473,39894,20201128144712,model_1,04.16.3553,5,NaN


In [48]:
test_err[(test_err['user_id']==30820) & (test_err['model_nm']=='model_2') 
& (test_err['fwver']=='04.33.1261') 
& (test_err['errtype']==5) & (test_err['time']==20201115044317)]

,user_id,time,model_nm,fwver,errtype,errcode
937967,30820,20201115044317,model_2,04.33.1261,5,NaN
937968,30820,20201115044317,model_2,04.33.1261,5,40053


In [49]:
test_err.iloc[937967,5] = '40053'

In [50]:
test_err[(test_err['user_id']==33681) & (test_err['model_nm']=='model_2') & (test_err['fwver']=='04.33.1185') & (test_err['errtype']==5) & (test_err['time']==20201103110259)]

,user_id,time,model_nm,fwver,errtype,errcode
4038892,33681,20201103110259,model_2,04.33.1185,5,NaN
4038893,33681,20201103110259,model_2,04.33.1185,5,40053


In [51]:
test_err.iloc[4038892, 5] = "40053"

In [53]:
test_err[(test_err['user_id']==38991) & (test_err['model_nm']=='model_2') 
& (test_err['fwver']=='04.33.1261') & (test_err['errtype']==5) 
& (test_err['time']==20201127213838)]

,user_id,time,model_nm,fwver,errtype,errcode
9486880,38991,20201127213838,model_2,04.33.1261,5,40053
9486881,38991,20201127213838,model_2,04.33.1261,5,NaN


In [54]:
test_err.iloc[9486881, 5] = "40053"

In [55]:
test_err[(test_err['user_id']==39894) & (test_err['model_nm']=='model_1') 
& (test_err['fwver']=='04.16.3553') & (test_err['errtype']==5) 
& (test_err['time']==20201128144712)]

,user_id,time,model_nm,fwver,errtype,errcode
10425472,39894,20201128144712,model_1,04.16.3553,5,-1010
10425473,39894,20201128144712,model_1,04.16.3553,5,NaN


In [56]:
test_err.iloc[10425473, 5] = "-1010"

In [57]:
test_err.isnull().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     0
dtype: int64

### Test Quality Data Missing Value
- Fwver 결측치 존재
- quality_0, quality_1, quality_2, quality_5 결측치 존재


In [58]:
test_qual.isnull().sum()

time               0
user_id            0
fwver          22764
quality_0     106584
quality_1         11
quality_2      21115
quality_3          0
quality_4          0
quality_5         44
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [61]:
fwms_idx = test_qual[test_qual['fwver'].isnull()].index
test_qual = test_qual.drop(fwms_idx)


In [62]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].fillna(test_qual[i].mode(0)[0])

In [63]:
print(train_err.isnull().sum().sum(), end=", ")
print(test_err.isnull().sum().sum(), end=", ")
print(train_qual.isnull().sum().sum(), end=", ")
print(test_qual.isnull().sum().sum())

0, 0, 0, 0


### 데이터 기본 전처리
1. 변수 타입 통일
- time 변수 타입을 int에서 datetime으로 변경

In [65]:
# time 변수 타입 변경 이전 데이터 저장
train_err.to_csv(save_path + "train_err_data.csv", index=False)
train_qual.to_csv(save_path + "train_quality_data.csv", index=False)
train_prob.to_csv(save_path + "train_problem_data.csv", index=False)

test_err.to_csv(save_path + "test_err_data.csv", index=False)
test_qual.to_csv(save_path + "test_quality_data.csv", index=False)

In [69]:
def make_datetime(x): # datetime 데이터로 변환
        x = str(x)
        year  = int(x[:4])
        month = int(x[4:6])
        day   = int(x[6:8])
        hour  = int(x[8:10])
        minute  = int(x[10:12])
        sec  = int(x[12:])
        return dt.datetime(year, month, day, hour, minute, sec)

In [70]:
train_err.time = train_err.time.apply(lambda x : make_datetime(x))

In [71]:
train_qual = train_qual.sort_values(['user_id','time']).reset_index(drop=True)
train_qual.time = train_qual.time.apply(lambda x : make_datetime(x))

In [72]:
train_prob = train_prob.sort_values(['user_id','time']).reset_index(drop=True)
train_prob.time = train_prob.time.apply(lambda x : make_datetime(x))

In [73]:
test_err.time = test_err.time.apply(lambda x : make_datetime(x))

In [74]:
test_qual.time = test_qual.time.apply(lambda x : make_datetime(x))

In [75]:
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788539,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
788540,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
788541,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
788542,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [76]:
train_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788544 entries, 0 to 788543
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        788544 non-null  datetime64[ns]
 1   user_id     788544 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   788544 non-null  float64       
 4   quality_1   788544 non-null  int64         
 5   quality_2   788544 non-null  float64       
 6   quality_3   788544 non-null  int64         
 7   quality_4   788544 non-null  int64         
 8   quality_5   788544 non-null  object        
 9   quality_6   788544 non-null  int64         
 10  quality_7   788544 non-null  object        
 11  quality_8   788544 non-null  object        
 12  quality_9   788544 non-null  object        
 13  quality_10  788544 non-null  object        
 14  quality_11  788544 non-null  int64         
 15  quality_12  788544 non-null  int64         
dtypes:

In [77]:
def str2int(x):
    if type(x) == str:
        x = x.replace(",","")
        x = int(x)
        return x
    else:
        x = int(x)
        return x

In [78]:
for i in train_qual.columns[3:]:
    train_qual[i] = train_qual[i].apply(lambda x : str2int(x))

In [79]:
for i in test_qual.columns[3:]:
    test_qual[i] = test_qual[i].apply(lambda x : str2int(x))

### 이상치 확인
1. fwver의 이상치 확인

In [80]:
temp = train_err[['time','fwver']].groupby("fwver").time.unique()
df = pd.DataFrame(columns = ['fwver','start','end'])
for i in range(len(temp)):
    temp.values[i].sort()
    df=df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
                 ignore_index = True)

/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel_30969/907898699.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel_30969/907898699.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel_30969/907898699.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.Series([temp.index[i], temp.values[i][0], temp.values[i][-1]], index=df.columns),
/var/folders/72/222764852597cbmznbjd4bsc0000gn/T/ipykernel

In [81]:
def find_abnormal(x): # x는 살펴볼 기간 차이
    for fwver, st, ed in tqdm(df.values):
        if((ed - st).days < x):
            user = train_err[train_err['fwver'] == fwver].user_id.unique()
            print("fwver : {}, per_num : {}, period : {}, id : {}".format(fwver,len(user),(ed-st).days,user))

In [85]:
find_abnormal(3) # fwver : 펌웨어 버전 # per_num : 사용한 유저의 수 # id : 사용한 유저의 id

 30%|██▉       | 11/37 [00:00<00:01, 15.91it/s]

fwver : 04.22.1656, per_num : 1, period : 0, id : [19831]


 73%|███████▎  | 27/37 [00:01<00:00, 23.38it/s]

fwver : 05.15.2090, per_num : 1, period : 0, id : [18142]


100%|██████████| 37/37 [00:01<00:00, 21.25it/s]

fwver : 05.15.2092, per_num : 1, period : 0, id : [24279]
